In [1]:
! pip install --upgrade pip

In [2]:
! pip install openai langchain==0.0.239

In [3]:
import os
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import openai

In [4]:
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
#os.environ["LANGCHAIN_API_KEY"] = "-"
#os.environ["LANGCHAIN_PROJECT"] = "-"

In [5]:
df = pd.read_csv('../data/clean_mtsamples_data.csv')
df.head()

,patient_id,patient_name,transcription
0,PID00000,Eleanor Pires,"SUBJECTIVE:, This 23-year-old white female pr..."
1,PID00001,Michael Dobek,"PAST MEDICAL HISTORY:, He has difficulty climb..."
2,PID00002,Leigh Doyle,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ..."
3,PID00003,Laura Troxell,"2-D M-MODE: , ,1. Left atrial enlargement wit..."
4,PID00004,Doloris Moscato,1. The left ventricular cavity size and wall ...


In [6]:
#AZURE_OPENAI_KEY = "-"
#AZURE_OPENAI_ENDPOINT = "-"
#AZURE_ENGINE_NAME = "-"

openai.api_type = "azure"
openai.api_base = AZURE_OPENAI_ENDPOINT
openai.api_version = "2023-05-15"
openai.api_key = AZURE_OPENAI_KEY

In [13]:
llm = ChatOpenAI(
    temperature=0,
    openai_api_key=AZURE_OPENAI_KEY,
    engine=AZURE_ENGINE_NAME,
    deployment_id= AZURE_OPENAI_ENDPOINT
    )

ValidationError: ignored

In [8]:
template = """You will act as an expert in pharmacology and medicine.
Given a doctors notes you will extract all the medication mentioned, with respective dosage and frequency of intake.
If no medication is found output: "No medication found in this registry."
If medication is found create a markdown table with the medication information containing the following columns: Medication; Dosage; Frequency.
Empty fields in the markdown table should be field with -"""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)

first_human_shot_learning = "Regular physical activity and a balanced diet are essential for maintaining good health and preventing various chronic conditions."
first_ai_shot_learning = "No medication found in this registry."
secound_human_shot_learning = "Recommend starting with a daily insulin dosage and gradually adjusting it based on your blood sugar levels and individual needs."
secound_ai_shot_learning = "| Medication | Dosage | Frequency  |\n|------------|--------|------------|\n|    Xanax   |  10mg  | Once a day |\n| Cetirizine |    -   |      -     |"

first_human_prompt = HumanMessagePromptTemplate.from_template(first_human_shot_learning)
first_ai_prompt =     AIMessagePromptTemplate.from_template(first_ai_shot_learning)
secound_human_prompt = HumanMessagePromptTemplate.from_template(secound_human_shot_learning)
secound_ai_prompt =     AIMessagePromptTemplate.from_template(secound_ai_shot_learning)

human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [9]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        system_message_prompt,
        first_human_prompt,
        first_ai_prompt,
        secound_human_prompt,
        secound_ai_prompt,
        human_message_prompt])

In [10]:
chain = LLMChain(llm=llm, prompt=chat_prompt)

In [11]:
answer = chain.run(df.loc[0, 'transcription'])
print(answer)

|   Medication   | Dosage |   Frequency   |
|----------------|--------|---------------|
| Ortho Tri-Cyclen |   -    |       -       |
|     Allegra     |   -    |       -       |
|     Zyrtec      |   -    |       -       |
|   Loratadine    |   -    |       -       |
|    Nasonex     | 2 sprays in each nostril | Three weeks |


In [12]:
df.loc[0, 'transcription']

'SUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.  She used to have allergies when she lived in Seattle but she thinks they are worse here.  In the past, she has tried Claritin, and Zyrtec.  Both worked for short time but then seemed to lose effectiveness.  She has used Allegra also.  She used that last summer and she began using it again two weeks ago.  It does not appear to be working very well.  She has used over-the-counter sprays but no prescription nasal sprays.  She does have asthma but doest not require daily medication for this and does not think it is flaring up.,MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: , She has no known medicine allergies.,OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78.,HEENT:  Her throat was mildly erythematous without exudate.  Nasal mucosa was erythematous and swollen.  Only clear drainage was seen.  TMs were clear.,Neck:  Supple without adenopathy.,